In [10]:
import pandas as pd
import numpy as np
import itertools
import re
import weightedcalcs as wc

In [11]:
lead_data = pd.read_csv("tractnc2015.csv")

In [12]:
lead_data.head()

,TRACT,PUMA10,COUNTY,COUSUB,ROW INDEX,YBL INDEX,BLD INDEX,HFL INDEX,OWNER 0-30%,OWNER 30-50%,...,COUNTYFP,ELEP ADJ,GASP ADJ,UGASP CAL,UGASP HU,ELEP,ELEP CAL,GASP,GASP CAL,FULP
0,3.700102e+10,1600.0,1.0,3.700200e+09,63.0,0.0,7.0,0.0,0.0,0.0,...,37001,0.925086,1.21072,777.347489,1390.306901,1541.764544,1215.605863,660.325026,799.468753,55.607695
1,3.700102e+10,1600.0,1.0,3.700200e+09,64.0,0.0,7.0,1.0,0.0,0.0,...,37001,0.925086,1.21072,777.347489,1390.306901,1541.764544,1215.605863,660.325026,799.468753,55.607695
2,3.700102e+10,1600.0,1.0,3.700200e+09,65.0,0.0,7.0,2.0,0.0,0.0,...,37001,0.925086,1.21072,777.347489,1390.306901,1541.764544,1215.605863,660.325026,799.468753,55.607695
3,3.700102e+10,1600.0,1.0,3.700200e+09,66.0,0.0,7.0,3.0,0.0,0.0,...,37001,0.925086,1.21072,777.347489,1390.306901,1541.764544,1215.605863,660.325026,799.468753,55.607695
4,3.700102e+10,1600.0,1.0,3.700200e+09,68.0,0.0,7.0,5.0,0.0,0.0,...,37001,0.925086,1.21072,777.347489,1390.306901,1541.764544,1215.605863,660.325026,799.468753,55.607695


In [13]:
lead_data.columns

Index(['TRACT', 'PUMA10', 'COUNTY', 'COUSUB', 'ROW INDEX', 'YBL INDEX',
       'BLD INDEX', 'HFL INDEX', 'OWNER 0-30%', 'OWNER 30-50%', 'OWNER 50-80%',
       'OWNER 80-100%', 'OWNER 100%+', 'RENTER 0-30%', 'RENTER 30-50%',
       'RENTER 50-80%', 'RENTER 80-100%', 'RENTER 100%+', 'OWNER HINCP 0-30%',
       'OWNER HINCP 30-50%', 'OWNER HINCP 50-80%', 'OWNER HINCP 80-100%',
       'OWNER HINCP 100%+', 'RENTER HINCP 0-30%', 'RENTER HINCP 30-50%',
       'RENTER HINCP 50-80%', 'RENTER HINCP 80-100%', 'RENTER HINCP 100%+',
       'OWNER ELEP 0-30%', 'OWNER ELEP 30-50%', 'OWNER ELEP 50-80%',
       'OWNER ELEP 80-100%', 'OWNER ELEP 100%+', 'RENTER ELEP 0-30%',
       'RENTER ELEP 30-50%', 'RENTER ELEP 50-80%', 'RENTER ELEP 80-100%',
       'RENTER ELEP 100%+', 'OWNER GASP 0-30%', 'OWNER GASP 30-50%',
       'OWNER GASP 50-80%', 'OWNER GASP 80-100%', 'OWNER GASP 100%+',
       'RENTER GASP 0-30%', 'RENTER GASP 30-50%', 'RENTER GASP 50-80%',
       'RENTER GASP 80-100%', 'RENTER GASP 100%+',

I believe to aggregate this data we will have to turn Income Level and Occupation type into their own columns, or find a way to do our math so that the columns with the same data types match.

So we need the primary key to be:

TRACT: Census Tract
PUMA10: Public Use Microdata Areas
COUNTY: County
COUSUB: County Subdivision
YBL INDEX: Year of building first construction (index, 0-5)
BLD INDEX: Number of units in each building (index, 0-8)
HFL INDEX: Primary heating fuel type (index, 0-8)
*OCCUPANCY*: Renter or Owner Occupied
*INCOME_STRATA*: Income level relative to poverty line

So all of the stratified columns such as OWNER ELEP 30-50% would instead just become ELEP, with OWNER represented in the OCCUPANCY column and 30-50% represented in the INCOME_STRATA column. We will have the following columns of data:

COUNT: Housing Unit Counts
HINC: Household Income ($/year)
ELEP: Electricity Expenditures ($/month)
GASP: Gas Expenditures ($/month)
FULP: Other Fuel Expenditures ($/month)

Then we can create energy burden for each row quite easily: ENERGY_BURDEN = 12 * (ELEP + GASP + FULP) /  HINC

And from this dataset we can calculate the weighted average housing burden in any area (TRACT, PUMA10, COUNTY, COUSUB) using the Housing Unit Counts (COUNT) as the weighting factor.

For aggregating by YBL, BLD, and HFL we can use the Housing Unit Counts (COUNT) to create a weighting factor just as we would when aggregating beyond OCCUPANCY and INCOME_STRATA.


In [14]:
primary_key = ["TRACT","PUMA10","COUNTY","COUSUB","YBL INDEX","BLD INDEX","HFL INDEX"]
occupancy_types = ["OWNER","RENTER"]
data_types = ["ELEP","GASP","FULP","HINCP",""]
income_strata = ["0-30%","30-50%","50-80%","80-100%","100%+"]

inputdata = [occupancy_types, data_types, income_strata]
data_columns = list(itertools.product(*inputdata))

data_columns = [re.sub(' +',' ',' '.join(x)) for x in data_columns]

lead_data["TRACT"] = lead_data["TRACT"].astype("int")
lead_data["COUSUB"] = lead_data["COUSUB"].astype("int")
lead_data["COUNTY"] = lead_data["COUNTY"].astype("int")
lead_data["PUMA10"] = lead_data["PUMA10"].astype("int")
lead_data["BLD INDEX"] = lead_data["BLD INDEX"].astype("int")
lead_data["YBL INDEX"] = lead_data["YBL INDEX"].astype("int")
lead_data["HFL INDEX"] = lead_data["HFL INDEX"].astype("int")

long_lead_data = lead_data.set_index(primary_key, inplace=False)[data_columns]

count_rename_dict = {' '.join(x):' '.join(y) for x,y in zip(list(itertools.product(*[occupancy_types, income_strata])), list(itertools.product(*[occupancy_types,["COUNT"], income_strata])))}

long_lead_data.rename(mapper=count_rename_dict, axis=1, inplace=True)
long_lead_data.columns = long_lead_data.columns.str.split(' ', expand=True)

long_lead_data.columns.rename(names=['OCCUPANCY TYPE','INCOME STRATA'], level=[0,2], inplace=True)

wide_lead_data = long_lead_data.stack(level=[0,2])

def calculate_energy_burden(df):
    return (12 * (df["ELEP"] + df["GASP"] + df["FULP"])) / df["HINCP"]

wide_lead_data["ENERGY BURDEN"] = calculate_energy_burden(wide_lead_data)

#wide_lead_data.fillna({'ENERGY_BURDEN':0}, inplace=True)
wide_lead_data.reset_index().head(20)

,TRACT,PUMA10,COUNTY,COUSUB,YBL INDEX,BLD INDEX,HFL INDEX,OCCUPANCY TYPE,INCOME STRATA,COUNT,ELEP,FULP,GASP,HINCP,ENERGY BURDEN
0,37001020100,1600,1,3700199999,0,7,0,OWNER,0-30%,0.000000e+00,0.0,0.0,0.0,0.0,NaN
1,37001020100,1600,1,3700199999,0,7,0,OWNER,100%+,0.000000e+00,0.0,0.0,0.0,0.0,NaN
2,37001020100,1600,1,3700199999,0,7,0,OWNER,30-50%,0.000000e+00,0.0,0.0,0.0,0.0,NaN
3,37001020100,1600,1,3700199999,0,7,0,OWNER,50-80%,0.000000e+00,0.0,0.0,0.0,0.0,NaN
4,37001020100,1600,1,3700199999,0,7,0,OWNER,80-100%,0.000000e+00,0.0,0.0,0.0,0.0,NaN
5,37001020100,1600,1,3700199999,0,7,0,RENTER,0-30%,7.522087e-05,0.0,0.0,0.0,0.0,NaN
6,37001020100,1600,1,3700199999,0,7,0,RENTER,100%+,5.244609e-03,0.0,0.0,0.0,0.0,NaN
7,37001020100,1600,1,3700199999,0,7,0,RENTER,30-50%,1.976263e-323,0.0,0.0,0.0,0.0,NaN
8,37001020100,1600,1,3700199999,0,7,0,RENTER,50-80%,1.101823e-03,0.0,0.0,0.0,0.0,NaN
9,37001020100,1600,1,3700199999,0,7,0,RENTER,80-100%,4.300051e-03,0.0,0.0,0.0,0.0,NaN


In [15]:
#np.ma.mean(final)

In [16]:
#fully_aggregated = wide_lead_data.groupby("TRACT").apply(lambda dfx: (dfx["ENERGY BURDEN"] * dfx["COUNT"]).sum() / dfx["COUNT"].sum())
#fully_aggregated.head()

In [17]:
#print(np.ma.masked_invalid(final).mean())
#print(np.isinf(final).sum())



len(wide_lead_data)

1906290

In [41]:
features = ['TRACT','YBL INDEX','BLD INDEX','HFL INDEX','OCCUPANCY TYPE','INCOME STRATA']

def weighted_average_df(df, weight_column, feature_columns, target_columns):
    #(X * WT).sum() / WT.sum()
    df['WEIGHT'] = df[weight_column] / df.groupby(feature_columns)[weight_column].transform('sum')
    df[cols] = df['WEIGHT'].values[:, None] * df[cols]
    df.groupby(feature_columns)[cols+[weight_column,'WEIGHT']].sum()
    
def weighted_average_df(df, weight_column, feature_columns, target_columns):
    #(X * WT).sum() / WT.sum()
    df["WEIGHT"] = (df[target_columns].multiply(df[weight_column],axis='index')) / sum(df[weight_column])
    df[target_columns] = df['WEIGHT'].values[:, None] * df[target_columns]
    return df.groupby(feature_columns)[target_columns+[weight_column,'WEIGHT']].sum()

def weighted_average_df(df, weight_column, feature_columns, target_columns):
    #(X * WT).sum() / WT.sum() instead of X * (WT / WT.sum())
    #(X * WT).sum() / WT.sum()
    return df.groupby(feature_columns).apply(lambda x: pd.Series([(x[v].multiply(x[weight_column])).sum().div(sum(x[weight_column])) for v in target_columns]))

def weighted_average_df(df, weight_column, feature_columns, target_columns):
    #(X * WT).sum() / WT.sum()
    df[target_columns] = (df[target_columns].multiply(df[weight_column], axis="index")).div(df.groupby(feature_columns)[weight_column].transform('sum'),axis='index')
    return df.groupby(feature_columns)[target_columns+[weight_column]].sum()

data_df = pd.DataFrame({'ind':['la','p','la','la','p','g','g','la'],
                        'dist':[10.,5.,7.,8.,7.,2.,5.,3.],
                        'diff':[0.54,3.2,8.6,7.2,2.1,1.,3.5,4.5],
                        'cas':[1.,2.,3.,4.,5.,6.,7.,8.]}) 

test_df = weighted_average_df(df=data_df, 
                                       weight_column="dist", 
                                       feature_columns=['ind'],
                                       target_columns=['cas','diff']
                                      )
print(test_df)

tract_aggregated = weighted_average_df(df=wide_lead_data, 
                                       weight_column="COUNT", 
                                       feature_columns=features,
                                       target_columns=['ELEP','FULP','GASP','HINCP']
                                      )


tract_aggregated["ENERGY BURDEN"] = calculate_energy_burden(tract_aggregated)
tract_aggregated.head(20)

          cas      diff  dist
ind                          
g    6.714286  2.785714   7.0
la   3.107143  4.882143  28.0
p    3.750000  2.558333  12.0


ELEP  \
TRACT       YBL INDEX BLD INDEX HFL INDEX OCCUPANCY TYPE INCOME STRATA         
37001020100 0         7         0         OWNER          0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   
                                          RENTER         0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   
                                1         OWNER          0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   
                                          RENTER         0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   

                                                                        FULP  \
TRACT       YBL INDEX BLD INDEX HFL INDEX OCCUPANCY TYPE INCOME STRATA         
37001020100 0         7         0         OWNER          0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   
                                          RENTER         0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   
                                1         OWNER          0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   
                                          RENTER         0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%         0.0   

                                                                        GASP  \
TRACT       YBL INDEX BLD INDEX HFL INDEX OCCUPANCY TYPE INCOME STRATA         
37001020100 0         7         0         OWNER          0-30%           0.0   
                                                         100%+           0.0   
                                                         30-50%          0.0   
                                                         50-80%          0.0   
                                                         80-100%      

In [45]:
tract_aggregated.dropna().tail(20)

ELEP  \
TRACT       YBL INDEX BLD INDEX HFL INDEX OCCUPANCY TYPE INCOME STRATA               
37199960400 5         0         2         RENTER         50-80%         220.574283   
                                                         80-100%        204.111600   
                                3         OWNER          0-30%          145.214294   
                                                         100%+          105.420148   
                                                         30-50%          92.857624   
                                                         50-80%         114.855419   
                                                         80-100%        133.526608   
                                          RENTER         0-30%          151.571590   
                                                         100%+           59.346323   
                                                         30-50%          78.947952   
                                                         50-80%         227.096228   
                                                         80-100%        287.741154   
                                5         OWNER          0-30%           59.200196   
                                                         100%+           95.714072   
                                                         30-50%          84.958899   
                                                         50-80%         129.887689   
                                                         80-100%        101.308199   
                                          RENTER         0-30%           50.837973   
                                                         100%+          102.560506   
                                                         80-100%         60.350927   

                                                                                FULP  \
TRACT       YBL INDEX BLD INDEX HFL INDEX OCCUPANCY TYPE INCOME STRATA                 
37199960400 5         0         2         RENTER         50-80%         1.009722e-09   
                                                         80-100%        1.013884e-09   
                                3         OWNER          0-30%          2.150696e+03   
                                                         100%+          1.167004e+03   
                                                         30-50%         9.381565e+02   
                                                         50-80%         1.210419e+03   
                                                         80-100%        1.138362e+03   
                                          RENTER         0-30%          1.033140e+03   
                                                         100%+          6.791824e+02   
                                                         30-50%         1.259485e+03   
                                                         50-80%         3.585596e+02   
                                                         80-100%        3.968412e+01   
                                5         OWNER          0-30%          1.100513e+02   
                                                         100%+          7.058008e+02   
                                                         30-50%         3.833243e+02   
                                                         50-80%         2.003556e+02   
                                                         80-100%        4.581303e+02   
                                          RENTER         0-30%          1.000000e-09   
                                                         100%+          2.551102e+02   
                                                         80-100%        1.221041e+03   

                                                                                GASP  \
TRACT       YBL INDEX BLD INDEX HFL INDEX OCCUPANCY TYPE INCOME STRATA                 
37199960400 5         0         2         RENTER         50-80%         

In [43]:
fully_aggregated = weighted_average_df(df=wide_lead_data,
                                      weight_column="COUNT",
                                      feature_columns=["TRACT"],
                                      target_columns=['ELEP','FULP','GASP','HINCP'])

fully_aggregated["ENERGY BURDEN"] = calculate_energy_burden(fully_aggregated)
fully_aggregated.tail(20)

,ELEP,FULP,GASP,HINCP,COUNT,ENERGY BURDEN
TRACT,,,,,,
37195001000,213.110021,59.084902,59.758919,49785.077239,712.0,0.080013
37195001100,194.517875,92.234217,82.950533,52606.510453,668.0,0.084332
37195001200,204.823797,66.443210,76.320335,52661.445888,1381.0,0.079205
37195001300,202.803863,104.232901,66.264561,55233.041615,2294.0,0.081104
37195001400,199.764295,35.942931,52.450098,58415.552264,3708.0,0.059195
37195001500,209.998436,101.887742,89.555892,63669.042088,2196.0,0.075662
37195001600,196.873949,89.324283,65.346636,53423.922350,1201.0,0.078963
37195001700,206.455446,69.791395,91.722600,49587.097146,1549.0,0.089048
37197050101,156.609983,140.904255,37.683265,52925.665405,2307.0,0.076000


In [46]:
fully_aggregated_2 = weighted_average_df(df=tract_aggregated,
                                      weight_column="COUNT",
                                      feature_columns=["TRACT"],
                                      target_columns=['ELEP','FULP','GASP','HINCP'])

fully_aggregated_2["ENERGY BURDEN"] = calculate_energy_burden(fully_aggregated_2)
fully_aggregated_2.tail(20)

,ELEP,FULP,GASP,HINCP,COUNT,ENERGY BURDEN
TRACT,,,,,,
37195001000,213.110021,59.084902,59.758919,49785.077239,712.0,0.080013
37195001100,194.517875,92.234217,82.950533,52606.510453,668.0,0.084332
37195001200,204.823797,66.443210,76.320335,52661.445888,1381.0,0.079205
37195001300,202.803863,104.232901,66.264561,55233.041615,2294.0,0.081104
37195001400,199.764295,35.942931,52.450098,58415.552264,3708.0,0.059195
37195001500,209.998436,101.887742,89.555892,63669.042088,2196.0,0.075662
37195001600,196.873949,89.324283,65.346636,53423.922350,1201.0,0.078963
37195001700,206.455446,69.791395,91.722600,49587.097146,1549.0,0.089048
37197050101,156.609983,140.904255,37.683265,52925.665405,2307.0,0.076000


In [47]:
((fully_aggregated['COUNT'] * fully_aggregated['ENERGY BURDEN']) / fully_aggregated['COUNT'].sum()).sum()
#weighted_average_df(df=fully_aggregated, weight_column="COUNT", feature_columns=["TRACT"], target_columns=["ENERGY BURDEN"])

0.05415984488825122

In [50]:
tract_aggregated.head(10000).reset_index().to_csv("tractnc2015_cleaned_shorttest.csv", index_label='index')
tract_aggregated.reset_index().to_csv("tractnc2015_cleaned.csv", index_label='index')
fully_aggregated.reset_index().to_csv("tractnc2015_aggregated.csv", index_label='index')

In [9]:
features = ['TRACT','YBL INDEX','BLD INDEX','HFL INDEX','OCCUPANCY TYPE','INCOME STRATA']

calc = wc.Calculator("COUNT")
nicely_grouped = wide_lead_data.groupby(features)
calc.mean(nicely_grouped, "ELEP")#.round().astype(int)

KeyboardInterrupt: 

In [ ]:
features = ['TRACT','YBL INDEX','BLD INDEX','HFL INDEX','OCCUPANCY TYPE','INCOME STRATA']

# Define a lambda function to compute the weighted mean:
#wm = lambda x: np.average(x, weights=df.loc[x.index, "adjusted_lots"])

# Define a dictionary with the functions to apply for a given column:
#f = {'adjusted_lots': ['sum'], 'price': {'weighted_mean' : wm} }

# Groupby and aggregate with your dictionary:
#df.groupby(["contract", "month", "year", "buys"]).agg(f)

#wm = lambda x: np.average(x, weights=x["COUNT"])
#wm = lambda dfx: (dfx["ENERGY BURDEN"] * dfx["COUNT"]).sum() / dfx["COUNT"].sum()
#wm = lambda dfx: np.average(dfx, weights=wide_lead_data.loc[dfx.index, "COUNT"])
wtavg = lambda x: np.ma.average(x, weights = wide_lead_data.loc[x.index, "COUNT"])

f = {'COUNT': ['sum'], 
     'ELEP': {'weighted_mean' : wtavg}, 
     'FULP': {'weighted_mean' : wtavg},
     'GASP': {'weighted_mean' : wtavg},
     'HINCP': {'weighted_mean' : wtavg},
     'ENERGY BURDEN': {'weighted_mean' : wtavg}}

#tract_aggregated = wide_lead_data.groupby(features).agg(f)


#tract_aggregated.columns = [' '.join(col).strip() for col in tract_aggregated.columns.values]
tract_aggregated.columns = tract_aggregated.columns.get_level_values(0)

tract_aggregated.head()

In [ ]:
fully_aggregated = tract_aggregated.groupby("TRACT").apply(wtavg)#.agg(f)

In [7]:
wide_lead_data.reset_index()

,TRACT,PUMA10,COUNTY,COUSUB,YBL INDEX,BLD INDEX,HFL INDEX,OCCUPANCY TYPE,INCOME STRATA,COUNT,ELEP,FULP,GASP,HINCP,ENERGY_BURDEN
0,37001020100,1600,1,3700199999,0,7,0,OWNER,0-30%,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
1,37001020100,1600,1,3700199999,0,7,0,OWNER,100%+,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
2,37001020100,1600,1,3700199999,0,7,0,OWNER,30-50%,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
3,37001020100,1600,1,3700199999,0,7,0,OWNER,50-80%,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
4,37001020100,1600,1,3700199999,0,7,0,OWNER,80-100%,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
5,37001020100,1600,1,3700199999,0,7,0,RENTER,0-30%,7.522087e-05,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
6,37001020100,1600,1,3700199999,0,7,0,RENTER,100%+,5.244609e-03,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
7,37001020100,1600,1,3700199999,0,7,0,RENTER,30-50%,1.976263e-323,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
8,37001020100,1600,1,3700199999,0,7,0,RENTER,50-80%,1.101823e-03,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN
9,37001020100,1600,1,3700199999,0,7,0,RENTER,80-100%,4.300051e-03,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN


In [18]:
suffix_regex = '([0-9]*\-[0-9]*%)|(100%\+)'
#long_lead_data = pd.wide_to_long(lead_data.head(500), stubnames=["OWNER","RENTER"], i=primary_key, j="INCOME STRATA", sep=' ', suffix=suffix_regex)
#pd.melt(long_lead_data, id_vars=primary_key, value_vars=None, var_name=None, value_name='value', col_level=None)
#long_lead_data.columns = long_lead_data.columns.str.split(' ', expand=True)

#long_lead_data.head(20)
#long_lead_data.melt()
long_lead_data.columns

Index(['RENTER HINCP 80-100%', 'RENTER GASP 100%+', 'OWNER GASP 50-80%',
       'ROW INDEX', 'OWNER HINCP 80-100%', 'OWNER FULP 100%+', 'OWNER 100%+',
       'OWNER GASP 100%+', 'HU_SUM', 'OWNER ELEP 0-30%', 'ELEP CAL',
       'OWNER ELEP 100%+', 'RENTER HINCP 0-30%', 'GASP', 'RENTER HINCP 50-80%',
       'OWNER GASP 30-50%', 'OWNER ELEP 50-80%', 'RENTER GASP 0-30%',
       'OWNER FULP 80-100%', 'OWNER HINCP 100%+', 'RENTER FULP 80-100%',
       'ELEP', 'RENTER GASP 30-50%', 'RENTER ELEP 50-80%', 'ELEP ADJ',
       'OWNER HINCP 50-80%', 'RENTER GASP 80-100%', 'OWNER GASP 80-100%',
       'OWNER FULP 0-30%', 'RENTER FULP 100%+', 'RENTER ELEP 0-30%',
       'RENTER FULP 30-50%', 'RENTER HINCP 100%+', 'UGASP CAL',
       'OWNER HINCP 0-30%', 'STATE', 'UGASP HU', 'OWNER FULP 50-80%',
       'RENTER ELEP 30-50%', 'OWNER ELEP 30-50%', 'RENTER FULP 50-80%',
       'RENTER FULP 0-30%', 'COUNTYFP', 'GASP ADJ', 'RENTER ELEP 80-100%',
       'OWNER HINCP 30-50%', 'RENTER HINCP 30-50%', 'OWNER GAS

In [11]:
#Create OCCUPANCY column
long_lead_data.tail(20)

RENTER HINCP 80-100%  \
TRACT        PUMA10 COUNTY COUSUB       YBL INDEX BLD INDEX HFL INDEX INCOME STRATA                         
3.700102e+10 1600.0 1.0    3.700200e+09 4.0       5.0       1.0       0-30%                           0.0   
                                                                      30-50%                          0.0   
                                                                      50-80%                          0.0   
                                                                      80-100%                         0.0   
                                                            2.0       0-30%                           0.0   
                                                                      30-50%                          0.0   
                                                                      50-80%                          0.0   
                                                                      80-100%                         0.0   
                                                            3.0       0-30%                           0.0   
                                                                      30-50%                          0.0   
                                                                      50-80%                          0.0   
                                                                      80-100%                         0.0   
                                                            5.0       0-30%                           0.0   
                                                                      30-50%                          0.0   
                                                                      50-80%                          0.0   
                                                                      80-100%                         0.0   
                                                            8.0       0-30%                           0.0   
                                                                      30-50%                          0.0   
                                                                      50-80%                          0.0   
                                                                      80-100%                         0.0   

                                                                                     RENTER GASP 100%+  \
TRACT        PUMA10 COUNTY COUSUB       YBL INDEX BLD INDEX HFL INDEX INCOME STRATA                      
3.700102e+10 1600.0 1.0    3.700200e+09 4.0       5.0       1.0       0-30%                        0.0   
                                                                      30-50%                       0.0   
                                                                      50-80%                       0.0   
                                                                      80-100%                      0.0   
                                                            2.0       0-30%                        0.0   
                                                                      30-50%                       0.0   
                                                                      50-80%                       0.0   
                                                                      80-100%                      0.0   
                                                            3.0       0-30%                        0.0   
                                                                      30-50%                       0.0   
                                                                      50-80%                       0.0   
                                                                      80-100%                      0.0   
                                                            5.0       0-30%                        0.0   
                                                                      30-50%                    

In [ ]:

new_columns = ["OCCUPANCY","INCOME STRATA"]
target_data = ["COUNT","HINC","ELEP","GASP","FULP"]

desired_columns = primary_key + target_data